In [2]:
import pandas as pd
import numpy as np

In [3]:
# First inspect the problem area
with open('all_tables.csv', 'r', encoding='latin1') as f:
    lines = f.readlines()
    print("Lines 85-95:")
    for i in range(85, 95):
        if i < len(lines):
            print(f"Line {i}: {repr(lines[i][:200])}")  # First 200 chars


Lines 85-95:
Line 85: '"",,,,,G,,86.0,91.7,99.6\n'
Line 86: '1.1.,,,,,,,,,\n'
Line 87: 'GER in ECED,,,,,B,%,93.3,97.3,102.1\n'
Line 88: '2,,,,,,,,,\n'
Line 89: '"",,,,,T,,89.6,94.5,101.0\n'
Line 90: '"",,,,,G,,66.9,75.5,70.3\n'
Line 91: '1.1. The enrolment rate of 4-year-old children,,,,,,,,,\n'
Line 92: '"",,,,,B,%,71.1,78.7,70.2\n'
Line 93: '3 in ECED,,,,,,,,,\n'
Line 94: '"",,,,,T,,69.0,77.1,70.2\n'


In [10]:
# First attempt - most likely to work
try:
    df = pd.read_csv('all_tables.csv', encoding='latin1', header=None,
                     engine='python', skipinitialspace=True)
    print(" Success with basic read!")
except Exception as e:
    print(f"Basic read failed: {e}")
    
    # Second attempt - more flexible
    try:
        df = pd.read_csv('all_tables.csv', encoding='latin1', header=None,
                        engine='python', skipinitialspace=True, 
                        quoting=3, escapechar='\\')
        print(" Success with flexible quoting!")
    except Exception as e:
        print(f"Flexible read failed: {e}")
        
        # Third attempt - manual parsing
        import csv
        data = []
        with open('all_tables.csv', 'r', encoding='latin1') as f:
            reader = csv.reader(f, skipinitialspace=True)
            for row in reader:
                data.append(row)
        
        # Create DataFrame with padded columns
        max_cols = max(len(row) for row in data)
        for row in data:
            row.extend([''] * (max_cols - len(row)))
        
        df = pd.DataFrame(data)
        print(" Success with manual parsing!")

Basic read failed: Expected 1 fields in line 91, saw 7
Flexible read failed: Expected 1 fields in line 37, saw 7
 Success with manual parsing!


In [6]:
#Quick inspection
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("First 5 rows:\n", df.head())
print("\nMIssing values", df.isnull().sum())


Shape: (1367, 33)
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
First 5 rows:
                                                   0  1  2  3  4  5  6  7  8   \
0  1.1 BACKGROUND ..................................                           
1  1.2 DATA RELIABILITY AND VALIDITY ...............                           
2  1.3 REAL PLAN INITIATIVES IN IEMIS ..............                           
3  1.4 PREPARATION OF THE REPORT ...................                           
4  CHAPTER 2:  EARLY CHILDHOOD EDUCATION AND DEVE...                           

  9   ... 23 24 25 26 27 28 29 30 31 32  
0     ...                                
1     ...                                
2     ...                                
3     ...                                
4     ...                                

[5 rows x 33 columns]

MIssing values 0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0

In [8]:
# Simple cleaning - keep only data rows
clean_data = []
for idx, row in df.iterrows():
    non_empty = [str(cell).strip() for cell in row if pd.notna(cell) and str(cell).strip() != '']
    if len(non_empty) > 1:
        clean_data.append(non_empty)

print(f"Original: {len(df)} rows")
print(f"Cleaned: {len(clean_data)} data rows")

# Save to new CSV
max_cols = max(len(row) for row in clean_data)
for row in clean_data:
    row.extend([''] * (max_cols - len(row)))

clean_df = pd.DataFrame(clean_data)
clean_df.to_csv('cleaned_data.csv', index=False, header=False, encoding='utf-8')
print("Saved cleaned_data.csv")

Original: 1367 rows
Cleaned: 1037 data rows
Saved cleaned_data.csv


In [9]:
# Just print all data rows with row numbers
for i, row in enumerate(clean_data):
    print(f"Row {i}: {row}")

Row 0: ['School Types', 'Total', 'Community', 'Institutional', 'Religious', '', '', '', '', '', '', '', '', '', '', '']
Row 1: ['ECED', '41,284', '33,470', '7,814', '527', '', '', '', '', '', '', '', '', '', '', '']
Row 2: ['Total Unit:', '35,447', '27,298', '8,149', '1,424', '', '', '', '', '', '', '', '', '', '', '']
Row 3: ['Basic (1-5)', '17,257', '15,322', '1,935', '1,165', '', '', '', '', '', '', '', '', '', '', '']
Row 4: ['Basic (6-8)', '6,648', '4,763', '1,885', '167', '', '', '', '', '', '', '', '', '', '', '']
Row 5: ['Basic (1-8)', '23,905', '20,085', '3,820', '1,332', '', '', '', '', '', '', '', '', '', '', '']
Row 6: ['Secondary (1-10, 6-10', 'or', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Row 7: ['6493', '3,401', '3,092', '58', '', '', '', '', '', '', '', '', '', '', '', '']
Row 8: ['Secondary (1-12)', '5049', '3,812', '1,237', '34', '', '', '', '', '', '', '', '', '', '', '']
Row 9: ['Secondary Level (9-12)', '11,542', '7,213', '4,329', '92', '', '', '', '